**Animal Dataset**

Resnet50

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [4]:
data_root = "/animals/animals/"

In [5]:
batch_size = 32
img_height = 224
img_width = 224

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

NotFoundError: Could not find directory /animals/animals/

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
input_t = tf.keras.Input(shape=(224, 224, 3))

In [ ]:
res_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=input_t
)

In [ ]:
for layer in res_model.layers[:143]:
    layer.trainable = False

In [ ]:
for i, layer in enumerate(res_model.layers):
    print(i, layer, "-",layer.trainable)

In [ ]:
model = tf.keras.Sequential()
model.add(res_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(90, activation='softmax'))
model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [ ]:
NUM_EPOCHS = 20

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS)

In [ ]:
result_batch = model.predict(train_ds)

In [ ]:
labels_path = '/kaggle/input/animal-image-dataset-90-different-animals/name of the animals.txt'
animal_labels = np.array(open(labels_path).read().splitlines())
len(animal_labels)

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)